In [1]:
import pandas as pd
from configparser import ConfigParser
import psycopg2


In [23]:

def config(filename='Streamlit_App\\database.ini', section='PostgreSQL'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	# execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [24]:
connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 14.1, compiled by Visual C++ build 1914, 64-bit',)
Database connection closed.


In [64]:
Temp_Activity_DB = pd.read_csv("RealTime_Test/PostgreSQL_ActivityLog_DB_Test.csv", delimiter=';')
Temp_Activity_DB

,InputID,dt,Date,Time,Comp-Well,Activity,Hook Treshold,Remarks,PIC,Section
0,1,14/02/2022 12:00,14/02/2022,12:00:00,KS ORKA--SMP AAE-08,MAKE UP BHA,61,Test Data,BM,17 1/2
1,2,14/02/2022 15:00,14/02/2022,15:00:00,KS ORKA--SMP AAE-08,TRIP IN,61,Test Data,BM,17 1/2
2,3,14/02/2022 18:28,14/02/2022,18:28:12,KS ORKA--SMP AAE-08,DRILL OUT CEMENT,61,Test Data,BM,17 1/2
3,4,14/02/2022 19:50,14/02/2022,19:50:00,KS ORKA--SMP AAE-08,DRILLING FORMATION,61,Test Data,BM,17 1/2
4,5,14/02/2022 20:32,14/02/2022,20:32:05,KS ORKA--SMP AAE-08,CIRCULATE HOLE CLEANING,61,Test Data,BM,17 1/2
5,6,14/02/2022 23:03,14/02/2022,23:03:45,KS ORKA--SMP AAE-08,DRILLING FORMATION,61,Test Data,BM,17 1/2
6,7,15/02/2022 00:38,15/02/2022,00:38:51,KS ORKA--SMP AAE-08,CIRCULATE HOLE CLEANING,61,Test Data,BM,17 1/2
7,8,15/02/2022 02:13,15/02/2022,02:13:52,KS ORKA--SMP AAE-08,WIPER TRIP,61,Test Data,BM,17 1/2
8,9,15/02/2022 04:20,15/02/2022,04:20:12,KS ORKA--SMP AAE-08,DRILLING FORMATION,61,Test Data,BM,17 1/2
9,10,16/02/2022 05:36,16/02/2022,05:36:08,KS ORKA--SMP AAE-08,CIRCULATE HOLE CLEANING,61,Test Data,BM,17 1/2


In [65]:
from IPython.display import display as displayDF
def InputActivity2Postgre(Input_DF):
    Table_Column = ['input_id',
    'dt',
    'date',
    'time',
    'comp',
    'well',
    'activity',
    'in_slip_treshold',
    'remarks',
    'pic',
    'section']

    Input_DF[['comp', 'well']] = Input_DF['Comp-Well'].str.split('--', 1, expand=True)
    Input_DF.rename(columns = {'Hook Treshold':'in_slip_treshold', 'InputID':'input_id'}, inplace = True)
    Input_DF.columns= Input_DF.columns.str.lower()
    
    # displayDF(Input_DF[Table_Column])
    return Input_DF[Table_Column]


ActivityInput_DB = InputActivity2Postgre(Temp_Activity_DB)
ActivityInput_DB

,input_id,dt,date,time,comp,well,activity,in_slip_treshold,remarks,pic,section
0,1,14/02/2022 12:00,14/02/2022,12:00:00,KS ORKA,SMP AAE-08,MAKE UP BHA,61,Test Data,BM,17 1/2
1,2,14/02/2022 15:00,14/02/2022,15:00:00,KS ORKA,SMP AAE-08,TRIP IN,61,Test Data,BM,17 1/2
2,3,14/02/2022 18:28,14/02/2022,18:28:12,KS ORKA,SMP AAE-08,DRILL OUT CEMENT,61,Test Data,BM,17 1/2
3,4,14/02/2022 19:50,14/02/2022,19:50:00,KS ORKA,SMP AAE-08,DRILLING FORMATION,61,Test Data,BM,17 1/2
4,5,14/02/2022 20:32,14/02/2022,20:32:05,KS ORKA,SMP AAE-08,CIRCULATE HOLE CLEANING,61,Test Data,BM,17 1/2
5,6,14/02/2022 23:03,14/02/2022,23:03:45,KS ORKA,SMP AAE-08,DRILLING FORMATION,61,Test Data,BM,17 1/2
6,7,15/02/2022 00:38,15/02/2022,00:38:51,KS ORKA,SMP AAE-08,CIRCULATE HOLE CLEANING,61,Test Data,BM,17 1/2
7,8,15/02/2022 02:13,15/02/2022,02:13:52,KS ORKA,SMP AAE-08,WIPER TRIP,61,Test Data,BM,17 1/2
8,9,15/02/2022 04:20,15/02/2022,04:20:12,KS ORKA,SMP AAE-08,DRILLING FORMATION,61,Test Data,BM,17 1/2
9,10,16/02/2022 05:36,16/02/2022,05:36:08,KS ORKA,SMP AAE-08,CIRCULATE HOLE CLEANING,61,Test Data,BM,17 1/2


In [70]:
# Define a function that handles and parses psycopg2 exceptions
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()    
    # get the line number when exception occured
    line_n = traceback.tb_lineno    
    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type) 
    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)    
    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")
def InsertBulk_Activity(conn, datafrm, table):
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(datafrm.columns))
    
    # SQL query to execute
    sql = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    print(sql)
    cursor = conn.cursor()
    try:
        cursor.executemany(sql, tpls)
        print("Data inserted using execute_many() successfully...")
    except (Exception, psycopg2.DatabaseError) as err:
        # pass exception to function
        show_psycopg2_exception(err)
        cursor.close()


params = config()

# connect to the PostgreSQL server
# print('Connecting to the PostgreSQL database...')
# conn = psycopg2.connect(**params)
conn = psycopg2.connect(**params)

InsertBulk_Activity(conn, ActivityInput_DB, 'activitylog_db')
conn.close()

INSERT INTO activitylog_db(input_id,dt,date,time,comp,well,activity,in_slip_treshold,remarks,pic,section) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
Data inserted using execute_many() successfully...


In [94]:
import psycopg2
import pandas as pd

# Connection parameters, yours will be different
# host=localhost
# database=PDU_AUTOMAPPING
# user=postgres
# password=Saber2496
param_dic = {
    "host"      : "localhost",
    "database"  : "PDU_AUTOMAPPING",
    "user"      : "postgres",
    "password"  : "Saber2496"
}

def connect_Postgre(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn


def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df
# Connect to the database
conn = connect_Postgre(param_dic)
Table_Column = ['input_id',
    'dt',
    'date',
    'time',
    'comp',
    'well',
    'activity',
    'in_slip_treshold',
    'remarks',
    'pic',
    'section']
# column_names = ["id", "source", "datetime", "mean_temp"]
# Execute the "SELECT *" query
df = postgresql_to_dataframe(conn, "select * from activitylog_db", Table_Column)
df.head()

Connecting to the PostgreSQL database...
Connection successful


,input_id,dt,date,time,comp,well,activity,in_slip_treshold,remarks,pic,section


In [103]:
def single_insert(conn, insert_req):
    """ Execute a single INSERT request """
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()
    
list_col='('
for i in Table_Column:
    list_col = list_col + i + ", "

list_col = list_col+')'
conn = connect_Postgre(param_dic)
for i in ActivityInput_DB.index:

    query = """
    INSERT into activitylog_db(input_id, dt, date, time, comp, well, activity, in_slip_treshold, remarks, pic, section) values(%s,'%s','%s','%s','%s','%s','%s',%s,'%s','%s','%s');
    """ % tuple(ActivityInput_DB.iloc[i,:].to_list())
    print(query)
    single_insert(conn, query)
# InsertBulk_Activity(conn, ActivityInput_DB, 'activitylog_db')
conn.close()

Connecting to the PostgreSQL database...
Connection successful

    INSERT into activitylog_db(input_id, dt, date, time, comp, well, activity, in_slip_treshold, remarks, pic, section) values(1,'14/02/2022 12:00','14/02/2022','12:00:00','KS ORKA','SMP AAE-08','MAKE UP BHA',61,'Test Data','BM','17 1/2');
    

    INSERT into activitylog_db(input_id, dt, date, time, comp, well, activity, in_slip_treshold, remarks, pic, section) values(2,'14/02/2022 15:00','14/02/2022','15:00:00','KS ORKA','SMP AAE-08','TRIP IN',61,'Test Data','BM','17 1/2');
    

    INSERT into activitylog_db(input_id, dt, date, time, comp, well, activity, in_slip_treshold, remarks, pic, section) values(3,'14/02/2022 18:28','14/02/2022','18:28:12','KS ORKA','SMP AAE-08','DRILL OUT CEMENT',61,'Test Data','BM','17 1/2');
    

    INSERT into activitylog_db(input_id, dt, date, time, comp, well, activity, in_slip_treshold, remarks, pic, section) values(4,'14/02/2022 19:50','14/02/2022','19:50:00','KS ORKA','SMP AAE-08','D

In [108]:
conn = connect_Postgre(param_dic)

df_get = postgresql_to_dataframe(conn, "select * from activitylog_db", Table_Column)
conn.close()
df_get



Connecting to the PostgreSQL database...
Connection successful


,input_id,dt,date,time,comp,well,activity,in_slip_treshold,remarks,pic,section
0,1,2022-02-14 12:00:00+08:00,2022-02-14,12:00:00,KS ORKA,SMP AAE-08,MAKE UP BHA,61,Test Data,BM,17 1/2
1,2,2022-02-14 15:00:00+08:00,2022-02-14,15:00:00,KS ORKA,SMP AAE-08,TRIP IN,61,Test Data,BM,17 1/2
2,3,2022-02-14 18:28:00+08:00,2022-02-14,18:28:12,KS ORKA,SMP AAE-08,DRILL OUT CEMENT,61,Test Data,BM,17 1/2
3,4,2022-02-14 19:50:00+08:00,2022-02-14,19:50:00,KS ORKA,SMP AAE-08,DRILLING FORMATION,61,Test Data,BM,17 1/2
4,5,2022-02-14 20:32:00+08:00,2022-02-14,20:32:05,KS ORKA,SMP AAE-08,CIRCULATE HOLE CLEANING,61,Test Data,BM,17 1/2
5,6,2022-02-14 23:03:00+08:00,2022-02-14,23:03:45,KS ORKA,SMP AAE-08,DRILLING FORMATION,61,Test Data,BM,17 1/2
6,7,2022-02-15 00:38:00+08:00,2022-02-15,00:38:51,KS ORKA,SMP AAE-08,CIRCULATE HOLE CLEANING,61,Test Data,BM,17 1/2
7,8,2022-02-15 02:13:00+08:00,2022-02-15,02:13:52,KS ORKA,SMP AAE-08,WIPER TRIP,61,Test Data,BM,17 1/2
8,9,2022-02-15 04:20:00+08:00,2022-02-15,04:20:12,KS ORKA,SMP AAE-08,DRILLING FORMATION,61,Test Data,BM,17 1/2
9,10,2022-02-16 05:36:00+08:00,2022-02-16,05:36:08,KS ORKA,SMP AAE-08,CIRCULATE HOLE CLEANING,61,Test Data,BM,17 1/2


In [110]:
"DELETE FROM activitylog_db where input_id = %s" % 1

'DELETE FROM activitylog_db where input_id = 1'

In [132]:
Well = 'SMP AAE-08'
Comp = 'KS ORKA'
select_query = "select * from activitylog_db"
select_query += " where (well = '%s' and comp = '%s')" % (Well, Comp)
conn = connect_Postgre(param_dic)

df_get = postgresql_to_dataframe(conn, select_query, Table_Column)
conn.close()
df_get

Connecting to the PostgreSQL database...
Connection successful


,input_id,dt,date,time,comp,well,activity,in_slip_treshold,remarks,pic,section


In [130]:
select_query

"select * from activitylog_db where (well = 'SMP AAE-08' and comp = 'KS ORKA')"

In [140]:
test_dict = {
    'a':1,
    'b':'aa',
    'c':'321',
}
tuple(test_dict.values())

(1, 'aa', '321')

In [149]:

def DeleteInputActivity_DB(Conn, Well, Comp, InputID):
    SQL_Queries = "delete from activitylog_db"
    SQL_Queries += " where ((well = '%s' and comp = '%s') and input_id = %s)" % (Well, Comp, InputID)

    
    cursor = Conn.cursor()
    try:
        cursor.execute(SQL_Queries)
        Conn.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        Conn.rollback()
        cursor.close()
        return 1
    cursor.close()
    Conn.close() 

Well = 'SMP AAE-08'
Comp = 'KS ORKA'
select_query = "select * from activitylog_db"
select_query += " where (well = '%s' and comp = '%s')" % (Well, Comp)

conn = connect_Postgre(param_dic)
df_get1 = postgresql_to_dataframe(conn, select_query, Table_Column)

DeleteInputActivity_DB(conn, Well, Comp, 17)

conn = connect_Postgre(param_dic)
df_get2 = postgresql_to_dataframe(conn, select_query, Table_Column)
conn.close()

Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful


In [151]:
df_get2

,input_id,dt,date,time,comp,well,activity,in_slip_treshold,remarks,pic,section
0,1,2022-02-14 12:00:00+08:00,2022-02-14,12:00:00,KS ORKA,SMP AAE-08,MAKE UP BHA,61,Test Data,BM,17 1/2
1,2,2022-02-14 15:00:00+08:00,2022-02-14,15:00:00,KS ORKA,SMP AAE-08,TRIP IN,61,Test Data,BM,17 1/2
2,3,2022-02-14 18:28:00+08:00,2022-02-14,18:28:12,KS ORKA,SMP AAE-08,DRILL OUT CEMENT,61,Test Data,BM,17 1/2
3,4,2022-02-14 19:50:00+08:00,2022-02-14,19:50:00,KS ORKA,SMP AAE-08,DRILLING FORMATION,61,Test Data,BM,17 1/2
4,5,2022-02-14 20:32:00+08:00,2022-02-14,20:32:05,KS ORKA,SMP AAE-08,CIRCULATE HOLE CLEANING,61,Test Data,BM,17 1/2
5,6,2022-02-14 23:03:00+08:00,2022-02-14,23:03:45,KS ORKA,SMP AAE-08,DRILLING FORMATION,61,Test Data,BM,17 1/2
6,7,2022-02-15 00:38:00+08:00,2022-02-15,00:38:51,KS ORKA,SMP AAE-08,CIRCULATE HOLE CLEANING,61,Test Data,BM,17 1/2
7,8,2022-02-15 02:13:00+08:00,2022-02-15,02:13:52,KS ORKA,SMP AAE-08,WIPER TRIP,61,Test Data,BM,17 1/2
8,9,2022-02-15 04:20:00+08:00,2022-02-15,04:20:12,KS ORKA,SMP AAE-08,DRILLING FORMATION,61,Test Data,BM,17 1/2
9,10,2022-02-16 05:36:00+08:00,2022-02-16,05:36:08,KS ORKA,SMP AAE-08,CIRCULATE HOLE CLEANING,61,Test Data,BM,17 1/2


In [152]:
conn = connect_Postgre(param_dic)

df_get = postgresql_to_dataframe(conn, "select * from summary_activity_db", Table_Column)
conn.close()
df_get

Connecting to the PostgreSQL database...
Connection successful


,input_id,dt,date,time,comp,well,activity,in_slip_treshold,remarks,pic,section


In [ ]:
'comp'
'well'
'time_start'
'time_end'
'duration_minutes'
'hole_depth'
'bit_depth'
'meterage_drilling'
'rotate_drilling_time'
'slide_drilling_time'
'reaming_time'
'connection_time'
'on_bottom_hours'
'stand_duration'
'label_subactivity'
'label_activity'
'stand_meterage_drilling'
'stand_durationx'
'stand_on_bottom'
'stand_group'
'pic'
'section'
'remarks'


In [155]:
# for i in ActivityInput_DB.index:

#     query = """
#     INSERT into activitylog_db(input_id, dt, date, time, comp, well, activity, in_slip_treshold, remarks, pic, section) values(%s,'%s','%s','%s','%s','%s','%s',%s,'%s','%s','%s');
#     """ % tuple(ActivityInput_DB.iloc[i,:].to_list())
#     print(query)
#     single_insert(conn, query)



SQL_Queries = "INSERT into activitylog_db("
SQL_Queries += ', comp'
SQL_Queries += ', well'
SQL_Queries += ', time_start'
SQL_Queries += ', time_end'

SQL_Queries += ', duration_minutes'
SQL_Queries += ', hole_depth'
SQL_Queries += ', bit_depth'
SQL_Queries += ', meterage_drilling'
SQL_Queries += ', rotate_drilling_time'
SQL_Queries += ', slide_drilling_time'
SQL_Queries += ', reaming_time'
SQL_Queries += ', connection_time'
SQL_Queries += ', on_bottom_hours'
SQL_Queries += ', stand_duration'

SQL_Queries += ', label_subactivity'
SQL_Queries += ', label_activity'
SQL_Queries += ', stand_meterage_drilling'

SQL_Queries += ', stand_durationx'
SQL_Queries += ', stand_on_bottom'
SQL_Queries += ', stand_group'

SQL_Queries += ', pic'
SQL_Queries += ', section'
SQL_Queries += ', remarks'

SQL_Queries += ') values('

SQL_Queries += "'%s'"
SQL_Queries += ", '%s'"
SQL_Queries += ", '%s'"
SQL_Queries += ", '%s'"

SQL_Queries += ", %s"
SQL_Queries += ", %s"
SQL_Queries += ", %s"
SQL_Queries += ", %s"
SQL_Queries += ", %s"
SQL_Queries += ", %s"
SQL_Queries += ", %s"
SQL_Queries += ", %s"
SQL_Queries += ", %s"
SQL_Queries += ", %s"

SQL_Queries += ", '%s'"
SQL_Queries += ", '%s'"
SQL_Queries += ", '%s'"

SQL_Queries += ", %s"
SQL_Queries += ", %s"
SQL_Queries += ", %s"

SQL_Queries += ", '%s'"
SQL_Queries += ", '%s'"
SQL_Queries += ", '%s'"
SQL_Queries += ")" % % tuple(ActivityInput_DB.iloc[i,:].to_list())

SQL_Queries = "INSERT into activitylog_db(, comp, well, time_start, time_end, duration_minutes, hole_depth, bit_depth, meterage_drilling, rotate_drilling_time, slide_drilling_time, reaming_time, connection_time, on_bottom_hours, stand_duration, label_subactivity, label_activity, stand_meterage_drilling, stand_durationx, stand_on_bottom, stand_group, pic, section, remarks) values(, '%s', '%s', '%s', '%s', %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, '%s', '%s', '%s', %s, %s, %s, '%s', '%s', '%s'"
SQL_Queries += ")" % % tuple(ActivityInput_DB.iloc[i,:].to_list())

"INSERT into activitylog_db(, comp, well, time_start, time_end, duration_minutes, hole_depth, bit_depth, meterage_drilling, rotate_drilling_time, slide_drilling_time, reaming_time, connection_time, on_bottom_hours, stand_duration, label_subactivity, label_activity, stand_meterage_drilling, stand_durationx, stand_on_bottom, stand_group, pic, section, remarks) values(, '%s', '%s', '%s', '%s', %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, '%s', '%s', '%s', %s, %s, %s, '%s', '%s', '%s'"

In [202]:

pd.to_datetime(summaryDB['date']).dt.date

0      2022-02-14
1      2022-02-14
2      2022-02-14
3      2022-02-14
4      2022-02-14
          ...    
152    2022-02-14
153    2022-02-14
154    2022-02-14
155    2022-02-14
156    2022-02-14
Name: date, Length: 157, dtype: object

In [214]:

summaryDB = pd.read_csv('RealTime_Test\\Temp_SummaryActivity - Copy.csv')
WellName = 'SMP AAE08'
CompName = 'KS ORKA'
PIC_Info = "-"
Remarks_Info = "-" 
Section_Info = "-"


def SummaryTranslator(summaryDB, WellName, CompName, PIC_Info, Remarks_Info, Section_Info):
    summaryDB['comp'] = WellName
    summaryDB['well'] = CompName
    summaryDB['pic'] = PIC_Info
    summaryDB['section'] = Remarks_Info
    summaryDB['remarks'] = Section_Info
    summaryDB['time_start'] = pd.to_datetime(summaryDB.date + " " + summaryDB.Time_start)
    summaryDB['time_end'] = pd.to_datetime(summaryDB.date + " " + summaryDB.Time_end)
    summaryDB['date'] = pd.to_datetime(summaryDB['date']).dt.date
    listcolumn = ['comp',
    'well',
    'time_start',
    'time_end',
    'duration_minutes',
    'hole_depth',
    'bit_depth',
    'meterage_drilling',
    'rotate_drilling_time',
    'slide_drilling_time',
    'reaming_time',
    'connection_time',
    'on_bottom_hours',
    'stand_duration',
    'label_subactivity',
    'label_activity',
    'stand_meterage_drilling',
    'stand_durationx',
    'stand_on_bottom',
    'stand_group',
    'pic',
    'section',
    'remarks']
    ColumnName_Dict = {
    'duration_minutes':'Duration(minutes)',
    'hole_depth':'Hole Depth(max)',
    'bit_depth':'Bit Depth(mean)',
    'meterage_drilling':'Meterage(m)(Drilling)',
    'rotate_drilling_time':'RotateDrilling',
    'slide_drilling_time':'Slide Drilling',
    'reaming_time':'ReamingTime',
    'connection_time':'ConnectionTime',
    'on_bottom_hours':'On Bottom Hours',
    'stand_duration':'Stand Duration',
    'label_subactivity':'LABEL_SubActivity',
    'label_activity':'LABEL_Activity',
    'stand_meterage_drilling':'Stand Meterage (m) (Drilling)',
    'stand_durationx':'Stand Stand Duration (hrs)',
    'stand_on_bottom':'Stand On Bottom Hours',
    'stand_group':'Stand Group_Pred'
    }
    ColumnName_Dict_fin = dict((y,x) for x,y in ColumnName_Dict.items())

    summaryDB.rename(columns = ColumnName_Dict_fin, inplace = True)
    return summaryDB[listcolumn]


summaryDB = SummaryTranslator(summaryDB, WellName, CompName, PIC_Info, Remarks_Info, Section_Info)

In [219]:
    for i in summaryDB.index:
        SQL_Queries = """INSERT into summary_activity_db(comp, well, time_start, time_end, duration_minutes, hole_depth, bit_depth, meterage_drilling, rotate_drilling_time, slide_drilling_time, reaming_time, connection_time, on_bottom_hours, stand_duration, label_subactivity, label_activity, stand_meterage_drilling, stand_durationx, stand_on_bottom, stand_group, pic, section, remarks) values(, '%s', '%s', '%s', '%s', %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, '%s', '%s', '%s', %s, %s, %s, '%s', '%s', '%s'"""
        SQL_Queries += ")" 

        print(SQL_Queries % tuple(summaryDB.iloc[i,:].to_list()))

INSERT into activitylog_db(, comp, well, time_start, time_end, duration_minutes, hole_depth, bit_depth, meterage_drilling, rotate_drilling_time, slide_drilling_time, reaming_time, connection_time, on_bottom_hours, stand_duration, label_subactivity, label_activity, stand_meterage_drilling, stand_durationx, stand_on_bottom, stand_group, pic, section, remarks) values(, 'SMP AAE08', 'KS ORKA', '2022-02-14 00:00:00', '2022-02-14 11:59:58', 719.97, 718.02, 640.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'FALSE/Check', 'nan', '0.0', 0.0, 0.0, nan, '-', '-', '-')
INSERT into activitylog_db(, comp, well, time_start, time_end, duration_minutes, hole_depth, bit_depth, meterage_drilling, rotate_drilling_time, slide_drilling_time, reaming_time, connection_time, on_bottom_hours, stand_duration, label_subactivity, label_activity, stand_meterage_drilling, stand_durationx, stand_on_bottom, stand_group, pic, section, remarks) values(, 'SMP AAE08', 'KS ORKA', '2022-02-14 12:00:03', '2022-02-14 14:59:56', 179.8

In [6]:
import pandas as pd
geodipa_1 = pd.read_csv("RealTime_Test\\26in_GEODIPA_PTH-V-7B_ActivitySummary.csv", delimiter=";")
geodipa_2 = pd.read_csv("RealTime_Test\GEODIPA_SLR-Q-31A_ActivitySummary (1).csv")
geodipa_1

,date,Time_start,Time_end,date_time,Duration(minutes),Hole Depth(max),Bit Depth(mean),Meterage(m)(Drilling),RotateDrilling,Slide Drilling,...,ConnectionTime,On Bottom Hours,Stand Duration,LABEL_SubActivity,LABEL_Activity,MERGE_SubActivity-Activity,Stand Meterage (m) (Drilling),Stand Stand Duration (hrs),Stand On Bottom Hours,Stand Group_Pred
0,1/19/2022,00:00:00,13:59:55,1/19/2022 0:00,839.92,0.00,0.00,0.0,0.0,0.0,...,0.0,0,0,FALSE/Check,NaN,nan--FALSE/Check,0.0,0.0,0.0,NaN
1,1/19/2022,14:00:00,00:00:00,1/19/2022 14:00,600.00,41.00,21.09,0.0,0.0,0.0,...,0.0,0,0,MAKE UP BHA,MAKE UP BHA,MAKE UP BHA--MAKE UP BHA,0.0,0.0,0.0,NaN
2,1/20/2022,00:00:00,01:04:55,1/20/2022 0:00,64.92,41.00,28.13,0.0,0.0,0.0,...,0.0,0,0,MAKE UP BHA,MAKE UP BHA,MAKE UP BHA--MAKE UP BHA,0.0,0.0,0.0,NaN
3,1/20/2022,01:05:00,01:19:40,1/20/2022 1:05,14.49,41.00,30.09,0.0,0.0,0.0,...,0.0,0,0,FALSE/Check,DRILLING FORMATION,DRILLING FORMATION--FALSE/Check,0.0,0.0,0.0,NaN
4,1/20/2022,01:19:45,01:23:10,1/20/2022 1:19,3.33,41.00,35.07,0.0,0.0,0.0,...,0.0,0,0,Wash Up/Down,DRILLING FORMATION,DRILLING FORMATION--Wash Up/Down,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,1/23/2022,09:54:12,09:59:57,1/23/2022 9:54,5.75,401.01,388.04,0.0,0.0,0.0,...,0.0,0,0,Stationary,TRIP IN,TRIP IN--Stationary,0.0,0.0,0.0,NaN
937,1/23/2022,10:00:02,10:06:47,1/23/2022 10:00,6.75,401.01,388.04,0.0,0.0,0.0,...,0.0,0,0,Stationary,CIRCULATE HOLE CLEANING,CIRCULATE HOLE CLEANING--Stationary,0.0,0.0,0.0,NaN
938,1/23/2022,10:06:52,12:29:57,1/23/2022 10:06,143.08,401.01,388.05,0.0,0.0,0.0,...,0.0,0,0,Wash Up/Down,CIRCULATE HOLE CLEANING,CIRCULATE HOLE CLEANING--Wash Up/Down,0.0,0.0,0.0,NaN
939,1/23/2022,12:30:02,15:29:57,1/23/2022 12:30,179.92,401.01,387.15,0.0,0.0,0.0,...,0.0,0,0,CEMENTING JOB,CEMENTING JOB,CEMENTING JOB--CEMENTING JOB,0.0,0.0,0.0,NaN


In [7]:
from Streamlit_App import Activity
geodipa_1_db = Activity.SummaryTranslator(geodipa_1, "PTH-V-7B", "GEODIPA", 'BM', "-", "26 in")
geodipa_2_db = Activity.SummaryTranslator(geodipa_2, "SLR-Q-31A", "GEODIPA", 'BM', "-", "-")


In [9]:
import psycopg2
def OpenConnection():
    param_dic = {
    "host"      : "localhost",
    "database"  : "PDU_AUTOMAPPING",
    "user"      : "postgres",
    "password"  : "Saber2496"
    }
    conn = None
    # import json


    
    # print(user_encode_data)
    # try:
        # connect to the PostgreSQL server
        # print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**param_dic)
    # except (Exception, psycopg2.DatabaseError) as error:
        # print(error)
        # sys.exit(1) 
    return conn
def InsertSummaryActivity_DB(Conn, SummaryActivity_DF):
    # SummaryActivity_DF = Activity.SummaryTranslator(SummaryActivity_DF, WellName, CompName, '-', '-', '-')
    for i in SummaryActivity_DF.index:
        SQL_Queries = """INSERT into summary_activity_db(comp, well, time_start, time_end, duration_minutes, hole_depth, bit_depth, meterage_drilling, rotate_drilling_time, slide_drilling_time, reaming_time, connection_time, on_bottom_hours, stand_duration, label_subactivity, label_activity, stand_meterage_drilling, stand_durationx, stand_on_bottom, stand_group, pic, section, remarks) values('%s', '%s', '%s', '%s', %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, '%s', '%s', '%s', %s, %s, '%s', '%s', '%s', '%s'"""
        SQL_Queries += ")" 

        # print()
        cursor = Conn.cursor()
        try:
            cursor.execute(SQL_Queries % tuple(SummaryActivity_DF.iloc[i,:].to_list()))
            Conn.commit()
        except(Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            Conn.rollback()
            cursor.close()
            return 1
        cursor.close()
    Conn.close() 

InsertSummaryActivity_DB(OpenConnection(), geodipa_1_db)
InsertSummaryActivity_DB(OpenConnection(), geodipa_2_db)


In [1]:
a={
'defaultColDef': {'minWidth': 5, 'editable': False, 'filter': True, 'resizable': True, 'sortable': True},
'columnDefs': [
    {'headerName': 'dt', 
    'field': 'dt', 
    'type': ['dateColumnFilter', 'shortDateTimeFormat'], 
    'checkboxSelection': True}, 
    {'headerName': 'Date', 
        'field': 'Date', 
        'type': []}, 
    {'headerName': 'Time', 
        'field': 'Time', 
        'type': []}, 
    {'headerName': 'Comp-Well', 
        'field': 'Comp-Well', 
        'type': []}, 
    {'headerName': 'Activity', 
        'field': 'Activity', 
        'type': [], 
        'editable': True, 
        'cellEditor': 'agSelectCellEditor', 
        'cellEditorPopup': True, 
        'cellEditorParams': {'values': ['DRILLING FORMATION', 'CIRCULATE HOLE CLEANING', 'CONNECTION', 'DRILL OUT CEMENT']}}, 
    {'headerName': 'Hook Treshold', 
        'field': 'Hook Treshold', 
        'type': ['numericColumn', 'numberColumnFilter']}, 
    {'headerName': 'Remarks', 
        'field': 'Remarks', 'type': []}, 
    {'headerName': 'PIC', 
        'field': 'PIC', 
        'type': []}
    ], 
'rowSelection': 'multiple', 
'rowMultiSelectWithClick': False, 
'suppressRowDeselection': False, 
'suppressRowClickSelection': True, 
'groupSelectsChildren': True, 
'groupSelectsFiltered': True
}

In [8]:
b = (a['columnDefs'][0])

In [5]:
print(1)

1
